# Attempt to download maps

In [ ]:
min_long = 37.52408
min_lat = 55.70015
max_long = 37.54093
max_lat = 55.70514
addr = "\"https://api.openstreetmap.org/api/0.6/map?bbox=" \
    + str(min_long)  + ',' + str(min_lat) + ',' + str(max_long) + ',' + str(max_lat) + '"'

!curl -o test_map.osm $addr

import shutil
import sys
!mv test_map.osm data
# with open('data/test_map.osm', encoding='utf-8') as file:
#     shutil.copyfileobj(file, sys.stdout)

In [ ]:
import codecs
import osm2geojson

with codecs.open('data/test_map.osm', 'r', encoding='utf-8') as data:
    xml = data.read()

geojson = osm2geojson.xml2geojson(xml, filter_used_refs=False, log_level='INFO')

gp.read_file(geojson)

[Parts of the world](http://download.geofabrik.de/)  
[Cities](https://download.bbbike.org/osm/bbbike/)  
[Choose borders](https://extract.bbbike.org/)  
osmnx  
[OSM API](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL) - possible solution

# Saving computed data

In [16]:
%%time
from visibility_graph import VisibilityGraph

# filename = "maps/kozlovo_extended.osm.pbf"
# bbox = [36.2, 56.5, 36.7, 57]
filename = "maps/kozlovo.osm.pbf"
bbox = [36.0, 56.45, 36.1, 56.5]

map_data = VisibilityGraph()
map_data.compute_geometry(filename=filename,
                          bbox=bbox,
                          epsilon_polygon=0.003,
                          epsilon_linestring=0.001,
                          bbox_comp=10)

# map_data.save_geometry("kozlovo_extended_362_565_367_57")
# map_data.save_geometry("kozlovo_36_5645_361_565")

# Using precomputed data

In [1]:
from visibility_graph import VisibilityGraph

map_data = VisibilityGraph()
# map_data.load_geometry("kozlovo_extended_362_565_367_57")
map_data.load_geometry("kozlovo_36_5645_361_565")

In [2]:
%%time
from geometry.supporting_convex import *
import mplleaflet

map_plot=('r', {0: "royalblue", 1: "r", 2: "k"})
G, fig = map_data.build_graph(find_pair,
                              inside_percent=0,
                              graph=True,
                              map_plot=None)

print('edges: ', G.number_of_edges())
print('nodes: ', G.number_of_nodes())
# mplleaflet.display(fig=fig)

# no need to fix overlay of edges (2-way paths)

edges:  3389
nodes:  283
Wall time: 33 s


In [2]:
%%time
from geometry.supporting_convex import *
import mplleaflet

map_plot=('r', {0: "royalblue", 1: "r", 2: "k"})
G, fig = map_data.build_graph(find_pair_cutoff,
                              inside_percent=0,
                              graph=True,
                              map_plot=None)

print('edges: ', G.number_of_edges())
print('nodes: ', G.number_of_nodes())
# mplleaflet.display(fig=fig)

# no need to fix overlay of edges (2-way paths)

edges:  2848
nodes:  288
Wall time: 31.1 s


# Find incident edges

In [9]:
import matplotlib.pyplot as plt
from geometry.supporting_convex import *
import mplleaflet

start = ([36.35, 56.57], None, None, None, None)
incidents = map_data.incident_vertices(start, find_pair_cutoff)
fig = plt.figure()
plt.scatter(start[0][0], start[0][1], color='r')
for p in incidents:
    plt.scatter(p[0][0], p[0][1], color='b')
mplleaflet.display(fig=fig)

# TODO
find pair O(log n), compare speed and keep the best without choise  
algorithm documentation to repo, license  
fix rare crosses  
downloading maps  